Creative Commons CC BY 4.0 Lynd Bacon & Associates, Ltd.  Not warranted to be suitable for any particular purpose. (You're on your own!)

## Validation Set Method for Regression

In the following we're going to verify the "out of sample" (test) predictive accuracy of a regression modeling using the _validation set_ method.  Let's first do some set-ups, and get some packages we're going to need.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt 

In [9]:
from sklearn import linear_model  # OLS 
from sklearn.metrics import mean_squared_error, r2_score # Basic metrics

# The Data and the Model

We're going to use the radon data in the file `radon.csv`.

 We'll regress `lcanmort` on:

* `lnradon`
* `obesity`
* `over65`
* `evrsmoke`
* `hhincome`

Let's get the data.  Let's read it in using a Pandas method:

In [2]:
radon=pd.read_csv('DATA/ML/radon.csv')
radon.columns

Index(['fips', 'state', 'county', 'lcanmort', 'radon', 'lnradon', 'obesity',
       'over65', 'cursmoke', 'evrsmoke', 'hhincome'],
      dtype='object')

## Dirty Little Secrets of Pandas

Pandas has several secrets.  Some are useful when needing to provide `scikit-learn` methods with data to be ingested.  Here's one of them.

Pandas Dataframes (and also Series and Panels) have an 'index' that labels rows.  A DataFrame can have a hierarchy of indexes.  (Indices?) You can  see what the default index values are for `radon` as `radon.index`.   

DataFrames, Series, and Panels have inside of them a numpy array, `numpy.ndarray`.  This makes them handy for use with `scikit-learn`, which likes to have data as np arrays as inputs.   

The data type of`radon.values` is numpy.ndarray.  As of a recent version of Pandas, the preferred method of getting the array out of a DataFrame is to use the `.to_numpy()` method. Let's do that here:

In [3]:
radonNumA=radon.to_numpy()
type(radonNumA)
radonNumA.shape

numpy.ndarray

(2881, 11)

Note that we can still know what the columns in this array are.  They are the same as the columns in the `radon` DataFrame.

## Validation Set Data Split

We're going to randomly split `radonNumA` into parts. An 80/20 split is a reasonable choice.  There are a couple of ways to do this.  Here's one of them, using `scikit-learn's`[train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) method:

In [4]:
from sklearn.model_selection import train_test_split

In [6]:
X, lcanmort = radonNumA[:,5:],radonNumA[:,3]

X_train, X_test, lcanmort_train, lcanmort_test = \
    train_test_split(X, lcanmort, test_size=0.20, random_state=99)

In [26]:
# Check those shapes:

X_train.shape
lcanmort_train.shape
X_test.shape
lcanmort_test.shape

(2304, 6)

(2304,)

(577, 6)

(577,)

Note that the parameter `random_state` sets a random number generator seed so that the results of the random split can be reproduced.   

Next, use X_train and lcanmort_train to "train" your regression model:

## Uh Oh.  Data Problem!

In [10]:
regr=linear_model.LinearRegression()  #creates an instance of the fcn
regr.fit(X_train,lcanmort_train)
regrPredYTrain=regr.predict(X_Train)  # values of y predicted by model
MSE=mean_squared_error(lcanmort_train,regrPredYTrain)  # calculated MSE
R2=r2_score(lcanmort_train,regrPredYTrain)  # coef of determination

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

Uh oh, (probably) some missing values.  Let's see how many there are, and where they are.  It's likely to be easiest to do this using Pandas, so let's go that way.  Let's create a DataFrame with just the variables we want to use for our regression modeling:

In [11]:
radon2=radon[['lcanmort', 'lnradon', 'obesity',
       'over65', 'cursmoke', 'evrsmoke', 'hhincome']]
radon2.columns

Index(['lcanmort', 'lnradon', 'obesity', 'over65', 'cursmoke', 'evrsmoke',
       'hhincome'],
      dtype='object')

Now find out how many there are on each variable:

In [12]:
radon2.isnull().sum()

lcanmort    0
lnradon     0
obesity     0
over65      0
cursmoke    0
evrsmoke    0
hhincome    1
dtype: int64

It appears that there is just one row of data with a missing value, a "np.nan," for the variable `hhincome`.  Let's just drop this row of data.  (Note that it would be worth examining this case, "just in case" there's something peculiar about it that's worth knowing about.)

## Munging

In [13]:
radon3=radon2.copy(deep=True).dropna()
radon3.shape
radon3.columns

(2880, 7)

Index(['lcanmort', 'lnradon', 'obesity', 'over65', 'cursmoke', 'evrsmoke',
       'hhincome'],
      dtype='object')

Looks good to go.  Back to the regression modeling.

In [14]:
radonNumA=radon3.to_numpy()
type(radonNumA)
radonNumA.shape

numpy.ndarray

(2880, 7)

In [15]:
X, lcanmort = radonNumA[:,1:],radonNumA[:,0]

X_train, X_test, lcanmort_train, lcanmort_test = \
    train_test_split(X, lcanmort, test_size=0.20, random_state=99)

In [16]:
# Check those shapes:

X_train.shape
lcanmort_train.shape
X_test.shape
lcanmort_test.shape

(2304, 6)

(2304,)

(576, 6)

(576,)

# Regression Again

In [17]:
regr=linear_model.LinearRegression()  #creates an instance of the fcn

regr.fit(X_train,lcanmort_train)

regrPredLCATrain=regr.predict(X_train)  # values of y predicted by model

MSEtrain=mean_squared_error(lcanmort_train,regrPredLCATrain)  # calculated MSE

R2train=r2_score(lcanmort_train,regrPredLCATrain)  # coef of determination

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Fit Metrics Based on Training Data

Fit metrics for the _training_ data:

In [18]:
print('Training MSE: %4.2f' % MSEtrain)
print('Training R2: %3.2f' % R2train)

Training MSE: 164.92
Training R2: 0.46


That's maybe a little sucky.  Just for giggles, let's see what the parameter estimates look like:

In [19]:
regr.coef_ # regression coefficients

array([-5.99564582,  0.83796963, -0.78949855,  1.03138729,  0.68959956,
        0.05226051])

In [20]:
regr.intercept_  # intercept coefficient

6.579975610912314

Note the scikit-learn convention of postpending an underscore on returned parameters.

Now let's see how the model fares at predicting `lcanmort` using the _test_ data:

In [21]:
regrPredLCATest=regr.predict(X_test)
MSEtest=mean_squared_error(lcanmort_test,regrPredLCATest)
R2test=r2_score(lcanmort_test,regrPredLCATest)

In [22]:
print('Test MSE: %4.2f' % MSEtest)
print('Test R2: %3.2f' % R2test)

Test MSE: 172.46
Test R2: 0.47


Not too bad "out of sample," it seems.

# A UD4U: Repeat the Above Using the Patient Satisfaction Data